In [1]:
import numpy as np
import torch

from tqdm.auto import tqdm
from pathlib import Path

import itertools

import os
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv(), verbose=True)

True

In [2]:
# results_dir = Path(os.getenv('RESULTS_DIR'))
results_dir = Path('/data/huze/ray_results/algonauts2021')
print('results_dir', results_dir)

finished_runs = [path.parent for path in results_dir.glob('**/prediction.npy')]
# print('finished_runs', len(finished_runs))

results_dir /data/huze/ray_results/algonauts2021


In [3]:
from pprint import pprint

exapmle_files = list(path.name for path in finished_runs[0].iterdir())
pprint(exapmle_files)

['params.pkl',
 'voxel_embedding.npy',
 'result.json',
 'hparams.yaml',
 'events.out.tfevents.1645601235.yfwu-guslab',
 'params.json',
 'progress.csv',
 'prediction.npy',
 'events.out.tfevents.1645601256.yfwu-guslab.3793893.0']


In [4]:
import yaml
from yaml import CLoader
import json
import pandas as pd
from src.config.config import flatten

finished_runs = [path.parent for path in results_dir.glob('**/prediction.npy')]

run_meta_infos = []
for run_dir in tqdm(finished_runs):
    hparams = yaml.load(run_dir.joinpath('hparams.yaml').open(), Loader=CLoader)
    run_meta_info = flatten(hparams)
    run_meta_info['path'] = run_dir

    data = [json.loads(line) for line in run_dir.joinpath('result.json').open()]
    ddf = pd.DataFrame(data)
    run_meta_info['score'] = ddf.val_corr.max()
    run_meta_info['time'] = ddf.time_total_s.max()

    run_meta_infos.append(run_meta_info)

run_df = pd.DataFrame(run_meta_infos)

# fix list unhashable
run_df['MODEL.BACKBONE.LAYERS'] = run_df['MODEL.BACKBONE.LAYERS'].apply(lambda x: tuple(x))
run_df['MODEL.NECK.SPP_LEVELS'] = run_df['MODEL.NECK.SPP_LEVELS'].apply(lambda x: tuple(x))

print(f'total GPU time {run_df.time.sum() / 3600:.2f}h')

  0%|          | 0/1496 [00:00<?, ?it/s]

total GPU time 279.05h


# prepare data

In [6]:
from src.utils.ensemble import optimize_val_correlation
from src.config.config import combine_cfgs, get_cfg_defaults
from src.data.datamodule import MyDataModule

In [7]:
# prepare validation data
cfg = combine_cfgs('../src/config/experiments/algonauts2021_i3d_flow.yml')
dm = MyDataModule(cfg)
dm.prepare_data()
dm.setup()

val_indices = dm.val_dataset.indices

cache_path = Path('/data_smr/huze/.cache/val_fmris.pt')

if cache_path.exists():
    val_fmris = torch.load(cache_path)
else:
    val_fmris = [dm.dataset_train_val.__getitem__(i)[1] for i in tqdm(val_indices)]
    val_fmris = np.stack(val_fmris, 0)
    val_fmris = torch.tensor(val_fmris)
    torch.save(val_fmris, cache_path)